# Dependências

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
!pip install gcloud
!gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=a821c4f78b3ae63b71dcc6972870863ff8d71d57415d9ab98269996547cd0b82
  Stored in directory: /root/.cache/pip/wheels/2a/62/75/3d74209bfebb8805823ae74afa28653aa1ea76d8b5a9d741ff
Successfully built gcloud
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=piVQY8gIthHWQky6K9PI1X6BrDdk75&prompt=consent&token_usage=remote&access_type=offline&code_chal

# Tratamento

In [ ]:
df = pd.read_csv('SIAPE_ativos_federal_raca_orgao_juridico.csv')
df

,ano,mes,mes_base,nat_jur,raça,vinculos
0,1999,2,Fev,Autarquia Federal,Branca,"87063,0"
1,1999,2,Fev,Autarquia Federal,Negra,"41211,0"
2,1999,2,Fev,Autarquia Federal,Não informado,"29267,0"
3,1999,2,Fev,Autarquia Federal,Outros,"6282,0"
4,1999,2,Fev,Emp Pub S A Cap Fechado,Branca,"379,0"
...,...,...,...,...,...,...
422,2024,2,Fev,Poder Executivo Federal,Negra,"69228,0"
423,2024,2,Fev,Poder Executivo Federal,Não informado,"260,0"
424,2024,2,Fev,Poder Executivo Federal,Outros,"3916,0"
425,2024,2,Fev,S A Cap Aberto Contr Estatal,Branca,"25,0"


In [ ]:
df['nat_jur'].unique()

array(['Autarquia Federal', 'Emp Pub  S A  Cap Fechado',
       'Fundacao Federal', 'Poder Executivo Federal',
       'S A Cap Aberto Contr Estatal'], dtype=object)

In [ ]:
df.drop(df[df['nat_jur'] == 'Emp Pub  S A  Cap Fechado'].index, inplace=True)
df.drop(df[df['nat_jur'] == 'S A Cap Aberto Contr Estatal'].index, inplace=True)

In [ ]:
df['nat_jur'].unique()

array(['Autarquia Federal', 'Fundacao Federal', 'Poder Executivo Federal'],
      dtype=object)

In [ ]:
df['mes_base'].unique()

array(['Fev'], dtype=object)

In [ ]:
df['raça'].unique()

array(['Branca', 'Negra', 'Não informado', 'Outros'], dtype=object)

In [ ]:
def transformar(nome):
    nome=re.sub(r"\bFev\b", "Fevereiro", nome)
    nome = re.sub(r"\bFundacao\b", "Fundação", nome)
    return nome

In [ ]:
df['mes_base'] =df['mes_base'].apply(transformar)

In [ ]:
df['nat_jur'] =df['nat_jur'].apply(transformar)

In [ ]:
df

,ano,mes,mes_base,nat_jur,raça,vinculos
0,1999,2,Fevereiro,Autarquia Federal,Branca,"87063,0"
1,1999,2,Fevereiro,Autarquia Federal,Negra,"41211,0"
2,1999,2,Fevereiro,Autarquia Federal,Não informado,"29267,0"
3,1999,2,Fevereiro,Autarquia Federal,Outros,"6282,0"
8,1999,2,Fevereiro,Fundação Federal,Branca,"36839,0"
...,...,...,...,...,...,...
420,2024,2,Fevereiro,Fundação Federal,Outros,"1534,0"
421,2024,2,Fevereiro,Poder Executivo Federal,Branca,"88192,0"
422,2024,2,Fevereiro,Poder Executivo Federal,Negra,"69228,0"
423,2024,2,Fevereiro,Poder Executivo Federal,Não informado,"260,0"


In [ ]:
df = df.rename(columns={'nat_jur': 'natureza_juridica', 'raça' : 'cor_raca', 'vinculos':'quantidade_vinculos'})


In [ ]:
df

,ano,mes,mes_base,natureza_juridica,cor_raca,quantidade_vinculos
0,1999,2,Fevereiro,Autarquia Federal,Branca,"87063,0"
1,1999,2,Fevereiro,Autarquia Federal,Negra,"41211,0"
2,1999,2,Fevereiro,Autarquia Federal,Não informado,"29267,0"
3,1999,2,Fevereiro,Autarquia Federal,Outros,"6282,0"
8,1999,2,Fevereiro,Fundação Federal,Branca,"36839,0"
...,...,...,...,...,...,...
420,2024,2,Fevereiro,Fundação Federal,Outros,"1534,0"
421,2024,2,Fevereiro,Poder Executivo Federal,Branca,"88192,0"
422,2024,2,Fevereiro,Poder Executivo Federal,Negra,"69228,0"
423,2024,2,Fevereiro,Poder Executivo Federal,Não informado,"260,0"


In [ ]:
df

,ano,mes,mes_base,natureza_juridica,cor_raca,quantidade_vinculos
0,1999,2,Fevereiro,Autarquia Federal,Branca,"87063,0"
1,1999,2,Fevereiro,Autarquia Federal,Negra,"41211,0"
2,1999,2,Fevereiro,Autarquia Federal,Não informado,"29267,0"
3,1999,2,Fevereiro,Autarquia Federal,Outros,"6282,0"
8,1999,2,Fevereiro,Fundação Federal,Branca,"36839,0"
...,...,...,...,...,...,...
420,2024,2,Fevereiro,Fundação Federal,Outros,"1534,0"
421,2024,2,Fevereiro,Poder Executivo Federal,Branca,"88192,0"
422,2024,2,Fevereiro,Poder Executivo Federal,Negra,"69228,0"
423,2024,2,Fevereiro,Poder Executivo Federal,Não informado,"260,0"


In [ ]:
df['cor_raca'].unique()

array(['Branca', 'Negra', 'Não informado', 'Outros'], dtype=object)

In [ ]:
df['quantidade_vinculos'] = df['quantidade_vinculos'].astype(str).str.replace(',', '.').astype(float).astype(int)

# Upload

In [ ]:
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('perfil_remuneracao')

/usr/local/lib/python3.12/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('mes_base','STRING',description='Mês de quando a observação foi executada'),
        bigquery.SchemaField('natureza_juridica','STRING',description='Natureza jurídica referente ao cargo da observação'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Número total de vinculos observados')]

In [ ]:
table_ref = dataset_ref.table('SIAPE_ativos_federal_raca_orgao_juridico_v3')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()


LoadJob<project=repositoriodedadosgpsp, location=US, id=81b0a78d-222a-46f1-9914-6e3905fd4e5f>